# Use Case #1

## Finding related images without pretraining, on toy data

The notebook below builds an MVP for this simple use case.

### User Story

The user provides an image to the system, and the number of similar images to be found in the Imagenette data set.

The system will return the requested number of images.

## TODO

0. Download the data using `!python '../src/data/get_imagenette.py'`, which will be stored at `../data/raw/imagenette-160`
1. create databunch (with `shuffle=False` for easy indexing later on)
2. create a `fastai` learner, based on pre-trained ResNet-18 (to be able to run on laptop).
3. create hook callback, pass model modules to be collected, create partial for learner creation
4. register callback with the learner by hand
5. use `.get_preds()` to collect activations for dataset
6. use `.pred()` on target image to collect activations for the target image
7. calculate distances from dataset activations to target image
8. get indices of closest activations
9. use the indices to retreive the closest images
10. plot the images
11. PROFIT!!!

## Imports

In [ ]:
from pathlib import Path
from fastai.vision import *
from fastai.metrics import accuracy
from fastai.callbacks.hooks import *

In [ ]:
DATA_PATH = '../data/raw/imagenette-160'
GET_DATA_PATH = '../src/data/get_imagenette.py'

## Getting the Data

In [ ]:
!ls ../src/data/

In [ ]:
# # $ allows to pass python variable to jupyter magic command
# !python $GET_DATA_PATH

In [ ]:
!ls $DATA_PATH

## Creating the Learner

In [ ]:
bs = 16
size = 160

data = (ImageList.from_folder(DATA_PATH)
        .use_partial_data(0.01)
        .split_none()
        .label_from_folder()
        .transform(size=size)
        .databunch(bs=bs, num_workers=0)
        .normalize(imagenet_stats))

data.show_batch(rows=4, figsize=(8,8))

In [ ]:
print(data.train_ds)
print('-'*42)
print(data.valid_ds)

# Creating the Learner and Collecting Activations
Callback hook will be registered with the learner and will allow to access activations of a given layer.

## Registering Callback by Hand

In [ ]:
class TestHook(HookCallback):
    def hook(self, m, i, o):
        pass
    def on_batch_end(self, train, **kwargs):
        print("Batch Done!")     
        
learner = cnn_learner(data, models.resnet18)

test_hook_callback = TestHook(learner)
learner.callbacks += [test_hook_callback]

learner.get_preds(data.train_ds)

---